## Calculate and save global integrals
This notebook calculates the global integral of biogeochemical quantities and saves them to netcdf files here: `/work/gam/projects/bio-pump-timescales/data/esm2m/processed/global_integrals_*/`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import bio_pump_timescales.esm2m as esm2m
import os
from dask.diagnostics import ProgressBar
import gfdl_utils as gu
import glob

In [3]:
grid = esm2m.load_grid()

### Global integrals (x,y,z)

In [11]:
variables = esm2m.disp_variables()
experiments = ['','_gat','_zero','_double']
override=False
# Directory to save file in
outroot = '/work/gam/projects/bio-pump-timescales/data/esm2m/processed/global_integrals/'
if override:
    outroot = outroot+'override-po4/'
else:
    outroot = outroot+'no-override/'
localdir = 'bling'
save = False

for variable in variables:
    # Ignore variables that have already been integrated
    if 'integral' in variable:
        continue
    for e in experiments:
        # Slight hack to deal with the fact that '' experiment
        # does not exist for ocean_gat_dic.
        if (e=='') & (esm2m.get_variable_dict()[variable]=='ocean_gat_dic'):
            continue
            
        outfile = 'esm2m_'+variable+e+'.nc'
        outpath = outroot+localdir+e+'/'+outfile
        if os.path.isfile(outpath):
            print(outfile+' exists.')
            continue
        print(outpath)
            
        # Load dataset
        ds = esm2m.load_exps(variable=variable,override=override,experiments=e)
        # Isolate variable
        da = ds[variable]
        dadims = list(da.dims)
        daattrs = da.attrs.copy()
        if 'st_ocean' in dadims:
            factor = 1035.0 * grid['volume_t']
            daattrs['units'] = daattrs['units']+' * kg'
        elif ['xt_ocean','yt_ocean'] in dadims:
            factor = grid['area_t']
            daattrs['units'] = daattrs['units']+' * m2'
        else:
            factor = 1
            
        # Remove time from list of dimensions before summing
        # Unless time is the only dimension
        if (len(dadims)==1) & (dadims[0]=='time'):
            # Already integrated
            dasum = da
        else:
            dadims.remove('time')
            dasum = ((da*factor).sum(dadims)
                     .assign_attrs(daattrs))
        dasum.name = variable
        
        if save:
            with ProgressBar():
                dasum.to_dataset().to_netcdf(outpath)

esm2m_alk.nc exists.
esm2m_alk_gat.nc exists.
esm2m_alk_zero.nc exists.
esm2m_alk_double.nc exists.
esm2m_alpha.nc exists.
esm2m_alpha_gat.nc exists.
esm2m_alpha_zero.nc exists.
esm2m_alpha_double.nc exists.
esm2m_biomass_p.nc exists.
esm2m_biomass_p_gat.nc exists.
esm2m_biomass_p_zero.nc exists.
esm2m_biomass_p_double.nc exists.
esm2m_chl.nc exists.
esm2m_chl_gat.nc exists.
esm2m_chl_zero.nc exists.
esm2m_chl_double.nc exists.
esm2m_co2_alpha.nc exists.
esm2m_co2_alpha_gat.nc exists.
esm2m_co2_alpha_zero.nc exists.
esm2m_co2_alpha_double.nc exists.
esm2m_co3_ion.nc exists.
esm2m_co3_ion_gat.nc exists.
esm2m_co3_ion_zero.nc exists.
esm2m_co3_ion_double.nc exists.
esm2m_delta_csurf.nc exists.
esm2m_delta_csurf_gat.nc exists.
esm2m_delta_csurf_zero.nc exists.
esm2m_delta_csurf_double.nc exists.
esm2m_delta_pco2.nc exists.
esm2m_delta_pco2_gat.nc exists.
esm2m_delta_pco2_zero.nc exists.
esm2m_delta_pco2_double.nc exists.
esm2m_dic.nc exists.
esm2m_dic_gat.nc exists.
esm2m_dic_zero.nc exis

### Global integrals, depth-resolved (x,y)

In [38]:
variables = cp.disp_variables()
experiments = ['','_gat','_zero','_double']
override=False
# Directory to save file in
outroot = '/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/'
outroot = cp.add_override_suffix(outroot,override)
localdir = 'bling'
    
for variable in variables:
    # Ignore variables that have already been integrated
    if ('integral' in variable) or ('flux' in variable) or ('stf' in variable):
        continue
    for e in experiments:
        outfile = 'esm2m_'+variable+e+'.nc'
        outpath = outroot+localdir+e+'/'+outfile
        if os.path.isfile(outpath):
            print(outfile+' exists.')
            continue
        print(outpath)
        
        # Load dataset
        ds = cp.load_exps(variable=variable,override=override,experiments=e)
        # Isolate variable
        da = ds[variable]
        dadims = list(da.dims)
        daattrs = da.attrs.copy()
        if ('st_ocean' not in dadims):
            print(variable+' is 2D.')
            continue
        else:
            factor = 1035.0 * grid['volume_t']
            daattrs['units'] = daattrs['units']+' * kg'
            
        # Remove time from list of dimensions before summing
        dadims.remove('time')
        dadims.remove('st_ocean')
        dasum = ((da*factor).sum(dadims)
                 .assign_attrs(daattrs))
        dasum.name = variable
        
        with ProgressBar():
            dasum.to_dataset().to_netcdf(outpath)

/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk.nc
[########################################] | 100% Completed | 22.3s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_gat.nc
[########################################] | 100% Completed | 23.0s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_zero.nc
[########################################] | 100% Completed | 25.9s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alk_double.nc
[########################################] | 100% Completed | 22.9s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alpha.nc
[########################################] | 100% Completed | 22.1s
/work/gam/projects/bio-pump-timescales/esm2m/data/global_integrals_depth-resolved/no-override/esm2m_alpha_g

### Spin-up

In [4]:
pathDict = {'pp':('/archive/Richard.Slater/Siena/'+
                  'siena_201308_rds-c3-gat-slurm/'+
                  'MOM5_SIS_BLING_CORE2-spinup-intel19/'+
                  'gfdl.ncrc3-intel19-prod-openmp/pp'),
           'ppname':'ocean_bling_ctrl_tracers',
           'out':'ts',
           'local':'monthly/20yr',
           'time':'*',
           'add':'jp_uptake_9999'}
path = gu.core.get_pathspp(**pathDict)
# glob.glob(path)

In [5]:
gu.core.issue_dmget(path)

0

In [6]:
ds = gu.core.open_frompp(**pathDict).chunk({'time':12})
# grid = esm2m.load_grid()

In [7]:
da = ds['jp_uptake_9999'].weighted(grid['area_t']).sum(['xt_ocean','yt_ocean'])

In [8]:
outroot = '/work/gam/projects/bio-pump-timescales/data/esm2m/processed/global_integrals/spinup/'
with ProgressBar():
    da.to_netcdf(outroot+'esm2m_jp_uptake_9999.nc')

[########################################] | 100% Completed | 15.7s


#### Recover atmospheric CO2

In [5]:
pathDict = {'pp':('/archive/Richard.Slater/Siena/'+
                  'siena_201308_rds-c3-gat-slurm/'+
                  'MOM5_SIS_BLING_CORE2-spinup-intel19/'+
                  'gfdl.ncrc3-intel19-prod-openmp/pp'),
           'ppname':'ocean_bling_ctrl_tracers',
           'out':'ts',
           'local':'monthly/20yr',
           'time':'*',
           'add':'delta_pco2'}
dpco2 = gu.core.open_frompp(**pathDict).chunk({'time':12})
pathDict['add'] = 'pco2_surf'
spco2 = gu.core.open_frompp(**pathDict).chunk({'time':12})

In [9]:
da = dpco2['delta_pco2']+spco2['pco2_surf']
da = da.weighted(grid['area_t']).mean(['xt_ocean','yt_ocean'])

In [11]:
outroot = '/work/gam/projects/bio-pump-timescales/data/esm2m/processed/global_integrals/spinup/'
with ProgressBar():
    da.to_netcdf(outroot+'esm2m_pco2_atm.nc')

[########################################] | 100% Completed | 28.4s


In [10]:
da

<xarray.DataArray (time: 11760)>
dask.array<truediv, shape=(11760,), dtype=float32, chunksize=(12,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 6561-01-16 12:00:00 ... 7540-12-16 12:00:00

In [17]:
outroot = '/work/gam/projects/bio-pump-timescales/data/esm2m/processed/global_integrals/spinup/'
with ProgressBar():
    da.to_netcdf(outroot+'esm2m_dic_stf.nc')

[########################################] | 100% Completed |  8.8s
